<div class="alert alert-block alert-info">
<b>INDEX:</b> this is a quick reference hyperlink to reach faster our results
</div>

<a id='section0'><b>Index</b></a>
    
<a href='#section1'>1- Link to the Libraries</a>

<a href='#section2'>2- Link to Global Variables</a>

<a href='#section3'>3- Link to the Functions</a>

<a href='#section4'>4- Link to Upload Files test</a>

<a href='#section5'>5- Link to Download Files test</a>

<a id='section1'>1- Link to Libraries</a>

In [ ]:
from __future__ import absolute_import, division, print_function

__author__ = "Agustin Bruzzese"
__copyright__ = "Copyright (C) 2020 Agustin Bruzzese"

__revision__ = "$Id$"
__version__ = "0.2"

import sys
sys.path.append("/usr/lib64/python3.6/site-packages/")

import io,json,linecache,logging,os,os.path,random,re,time,uuid,zipfile,string,pathlib,time,pytz,graphyte,socket
import numpy as np 
from urllib.parse import urlunsplit

from io import StringIO

# Set Rucio virtual environment configuration 
os.environ['RUCIO_HOME']=os.path.expanduser('~/rucio')
from rucio.rse import rsemanager as rsemgr
from rucio.client.client import Client
from rucio.client.didclient import DIDClient
from rucio.client.replicaclient import ReplicaClient
import rucio.rse.rsemanager as rsemgr
from rucio.client.ruleclient import RuleClient
from rucio.client.uploadclient import UploadClient
from rucio.client.downloadclient import DownloadClient
from rucio.common.utils import (adler32, detect_client_location, 
                                execute, generate_uuid, md5, 
                                send_trace, GLOBALLY_SUPPORTED_CHECKSUMS)
from rucio.common.exception import (AccountNotFound, Duplicate, RucioException, DuplicateRule, InvalidObject, DataIdentifierAlreadyExists, FileAlreadyExists, RucioException,
                                    AccessDenied, InsufficientAccountLimit, RuleNotFound, AccessDenied, InvalidRSEExpression,
                                    InvalidReplicationRule, RucioException, DataIdentifierNotFound, InsufficientTargetRSEs,
                                    ReplicationRuleCreationTemporaryFailed, InvalidRuleWeight, StagingAreaRuleRequiresLifetime)


import gfal2
from gfal2 import (
    Gfal2Context,
    GError,
)


<a id='section2'>2- Link to Global Variables </a>

In [ ]:
# Rucio settings 

## Remember to always have an alive and valid proxy before executing the script

## account user
account='root'
auth_type='x509_proxy'

CLIENT = Client(account=account, auth_type=auth_type)
didc = DIDClient(account=account, auth_type=auth_type)
repc = ReplicaClient(account=account, auth_type=auth_type)
client = Client(account=account, auth_type=auth_type)
uploadClient = UploadClient()
downloadClient = DownloadClient()

## user scope 
Default_Scope = 'test-root'

# Get list of all RSEs 
default_rses = list(client.list_rses())
rses_lists = []
for single_rse in default_rses :
    rses_lists.append(single_rse['rse'])

print(rses_lists)

# Gfal settings 
gfal = Gfal2Context()

print(json.dumps(client.whoami(), indent=4, sort_keys=True))
print(json.dumps(client.ping(), indent=4, sort_keys=True))

<a id='section3'>3- Link to Functions </a>

In [ ]:
# Global functions definition : 

def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('   EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))
    
def get_rse_url(rse):
    """
    Return the base path of the rucio url
    """
    rse_settings = rsemgr.get_rse_info(rse)
    protocol = rse_settings['protocols'][0]

    schema = protocol['scheme']
    prefix = protocol['prefix']
    port = protocol['port']
    rucioserver = protocol['hostname']

    if schema == 'srm':
        prefix = protocol['extended_attributes'][
            'web_service_path'] + prefix
    url = schema + '://' + rucioserver
    if port != 0:
        url = url + ':' + str(port)
    rse_url = url + prefix
    return(rse_url)

# Generate a random file : 

def generate_random_file(filename, size, copies = 1):
    """
    generate big binary file with the specified size in bytes
    :param filename: the filename
    :param size: the size in bytes
    :param copies: number of output files to generate
    
    """
    n_files = []
    n_files = np.array(n_files, dtype = np.float32)   
    for i in range(copies):
        file = filename + '-' + str(uuid.uuid4())
        if os.path.exists(file) : 
            print ("File %s already exist" %file)

        else:
            print ("File %s not exist" %file)    
            try : 
                newfile = open(file, "wb")
                newfile.seek(size)
                newfile.write(b"\0")
                newfile.close ()
                os.stat(file).st_size
                print('random file with size %f generated ok'%size)
                n_files = np.append(n_files, file)
            except :
                print('could not be generate file %s'%file)

    return(n_files)

<a href='#section4'>4- Link to Upload Files test</a>

In [ ]:
list_files = generate_random_file('deletion', 1, 2) 

if rses_lists is not None and list_files is not None : 
    for n in range(0, len(list_files)) :
        for rse in rses_lists: 

            client=Client()
            rulesClient=RuleClient()
            uploadClient=UploadClient()

            name_file = list_files[n]
            filePath="./"+name_file

            ## perform upload
            if client.get_rse(rse)["deterministic"] == True :
                try:
                    file = {'path': filePath, 'rse': rse, 'did_scope': Default_Scope, 'lifetime':1}
                    print(file)
                    uploadClient.upload([file])
                except:
                    PrintException()
            else :
                url_rse = os.path.join(get_rse_url(rse), Default_Scope, name_file)
                try:
                    file = {'path': filePath, 'rse': rse, 'did_scope': Default_Scope, 'lifetime':1, 'pfn':url_rse}
                    print(file)
                    uploadClient.upload([file])
                except:
                    PrintException()

        #os.remove(filePath)


<a href='#section5'>5- Link to Download Files test</a>

In [ ]:
# Download a file from the datalake.
for n in range(0, len(list_files)) :
    
    name_file = list_files[n]
    download = downloadClient.download_dids(items=[{'did':'{}:{}'.format(Default_Scope,name_file)}], num_threads=2, trace_custom_fields={}, traces_copy_out=None)

    print(download[0]['dest_file_paths'][0])
    os.remove(download[0]['dest_file_paths'][0]) 